<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/Youtube_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web スクレイピング と感情分析

【おことわり】演習は Yahoo リアルタイム検索でやりましたが、Youtube でやりたいというリクエストあったので作りました。流れは同じですが、検索するタグなどが異なります。



*   Web スクレイピング: Selenium --> https://ai-inter1.com/python-selenium/
*   感情分析： Transformer --> https://tt-tsukumochi.com/archives/4105



# 1. Selenium による Web スクレイピング

### Google Chrome, Chrom Driver, Selenium のインストール

In [ ]:
%%shell
# 更新を実行
sudo apt -y update

# ダウンロードのために必要なパッケージをインストール
sudo apt install -y wget curl unzip libvulkan1 libu2f-udev

# Chromeのインストール
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

# Chrome Driverのインストール
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://storage.googleapis.com/chrome-for-testing-public/131.0.6778.69/linux64/chromedriver-linux64.zip
#wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/126.0.6099.216/linux64/chromedriver-linux64.zip
unzip chromedriver-linux64.zip
chmod +x /content/chromedriver-linux64/chromedriver
mv /content/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver

pip install selenium

### ライブラリのインポート

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
import pandas as pd
from datetime import datetime, timedelta
import urllib.parse

from bs4 import BeautifulSoup
import time

### 検索語（クエリ）を設定

In [ ]:
url = 'https://www.youtube.com/watch?v=CkOuo1QbHWM'

### Chrome Driver の設定

In [ ]:
# ブラウザをheadlessモード実行
options = webdriver.ChromeOptions()
#ヘッドレスモード（バックグラウンドで起動）で実行。コラボの場合、必須。
options.add_argument('--headless')
#サンドボックスモードの解除。これも必須。
options.add_argument('--no-sandbox')
#これも設定した方がよい。
options.add_argument('--disable-dev-shm-usage')

### Web ページの取得

In [ ]:
#インスタンス化
driver = webdriver.Chrome(options=options)
#指定したドライバーが見つかるまで待機
driver.implicitly_wait(10)
# ページの取得
driver.get(url)

# 要素が表示されるまで待機
time.sleep(5)

### コメント数の確認

In [ ]:
comments = driver.find_elements(By.XPATH, '//*[@id="content-text"]/span')
print(len(comments), '個のコメントを取得')

### スクロールしてコメントを表示

In [ ]:
for j in range(500):
    print("\r%d/500" % j, end='')
    driver.execute_script('window.scrollBy(0, 100);')
    time.sleep(0.1)

comment = driver.find_elements(By.XPATH, '//*[@id="content-text"]/span')
print('\n', len(comment), '個のコメントを取得')

### 返信を展開して表示

In [ ]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

elem1 = driver.find_elements(By.XPATH, '//*[@id="more-replies"]/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]')
print(len(elem1), '個の要素を取得')
for i, x in enumerate(elem1):
    print('\r%d/%d' % (i, len(elem1)), end='')
    x.click()
    time.sleep(0.5)

### 返信に続きがある場合は展開して表示

In [ ]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

elem2 = driver.find_elements(By.XPATH, '//*[@id="button"]/ytd-button-renderer/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]')
print(len(elem2), '個の要素を取得')
for i, x in enumerate(elem2):
    print('\r%d/%d' % (i, len(elem2)), end='')
    x.click()
    time.sleep(0.5)

### コメント数の確認

In [ ]:
comments = driver.find_elements(By.XPATH, '//*[@id="content-text"]/span')
print("コメント数：", len(comments))

In [ ]:
comments[0].text

# 2. 感情分析

## WordCloud

### Mecab + Wordcloud のインストール

In [ ]:
!apt install mecab libmecab-dev mecab-ipadic-utf8
!ln -s /etc/mecabrc /usr/local/etc/mecabrc
!apt-get -y install fonts-ipafont-gothic
!pip install mecab-python3 wordcloud japanize-matplotlib

In [ ]:
import MeCab
import csv
import json
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import japanize_matplotlib
import warnings
import numpy as np
import pandas as pd
import collections

### Mecab の使い方

In [ ]:
mecab = MeCab.Tagger()
malist = mecab.parse("すもももももももものうち")
print(malist)

### 各コメントを単語に分割

In [ ]:
mecab = MeCab.Tagger()
sentences = []
for comment in comments:
    words = ""
    node = mecab.parseToNode(comment.text)
    while node:
        word = node.surface
        word_type = node.feature.split(",")[0]

        # "名詞", "動詞", "形容詞", "副詞"の中で選択したものを抽出
        if word_type in ["名詞"]:#, "動詞", "形容詞"]:
            words += ' ' + word
        node = node.next
    sentences.append(words)

In [ ]:
df = pd.DataFrame(sentences, columns=['text'])
df

### Wordcloud の作成

In [ ]:
#wordcloudで出力するフォントを指定
font_path = '/usr/share/fonts/opentype/ipafont-gothic/ipagp.ttf'
txt = " ".join(sentences)

# ストップワードの設定(意味のない単語が含まれないように設定)
stop_words = ['https', 't', 'co', 'RT', 'pic', 'com', 'twitter', 'jp']

#解析した単語、ストップワードを設定、背景の色は白
wordcloud = WordCloud(font_path=font_path, background_color="white", stopwords=set(stop_words), width=800, height=600).generate(txt)

fig = plt.figure(figsize=(30, 10))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

## ネガポジ分析

### Transformer のインストール

In [ ]:
!pip install transformers

### 感情分析関数（classifier）

In [ ]:
from transformers import pipeline

# パイプラインの準備
classifier = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
    return_all_scores=True
)

### 感情分析の使い方

In [ ]:
print(classifier("勇者ヒンメルならそうしました"))
print(classifier("ザクとは違うのだよ、ザクとは！"))
print(classifier("見ろ、人がゴミのようだ！！"))
print(classifier("おまえの物は俺の物、俺の物も俺の物"))

### 集計

In [ ]:
sentiments = []
for comment in comments:
    print(comment.text)
    senti = classifier(comment.text)
    print(senti)
    sentiments.append([senti[0][0]['score'], senti[0][1]['score'], senti[0][2]['score']])
print(sentiments)
print(np.mean(np.array(sentiments)))

In [ ]:
df = pd.DataFrame(data = np.array(sentiments), columns=['positive', 'negative', 'neutral'])
df

In [ ]:
df.sum().values

### グラフ化

In [ ]:
plt.figure(figsize=(8, 8))
plt.rcParams["font.size"] = 18
plt.pie(df.sum().values, labels=df.columns, counterclock=False, autopct='%0.1f%%', startangle=90)
plt.title(url + 'に対するネットの感情')
plt.show()